In [ ]:
import numpy as np

def find_nash_equilibrium(game_matrix):
    num_rows = game_matrix.shape[0]
    num_cols = game_matrix.shape[1]

    # Check if there exists a Nash equilibrium
    for row_strategy in range(num_rows):
        for col_strategy in range(num_cols):
            row_payoff = game_matrix[row_strategy, col_strategy, 0]
            col_payoff = game_matrix[row_strategy, col_strategy, 1]

            # Check if no player can improve their payoff by changing strategy
            if all(game_matrix[row_strategy, c, 1] <= col_payoff for c in range(num_cols)) and \
               all(game_matrix[r, col_strategy, 0] <= row_payoff for r in range(num_rows)):
                return True

    return False

def choose_starting_player(game_matrix):
    num_rows = game_matrix.shape[0]
    num_cols = game_matrix.shape[1]

    # Initialize strategies and cumulative average payoffs
    strategy_player1 = np.zeros(num_cols)  # Initial strategy for player 1
    strategy_player2 = np.zeros(num_rows)  # Initial strategy for player 2
    cum_payoffs_player1 = np.zeros(num_cols)
    cum_payoffs_player2 = np.zeros(num_rows)

    num_iterations = 1000  # Number of iterations for fictitious play

    for _ in range(num_iterations):
        # Compute best response for player 2 (assuming player 1's strategy)
        expected_payoffs_player2 = np.dot(game_matrix[:, :, 0], strategy_player1)
        best_response_player2 = np.argmax(expected_payoffs_player2)

        # Update strategy of player 2
        strategy_player2[best_response_player2] += 1

        # Compute best response for player 1 (assuming player 2's strategy)
        expected_payoffs_player1 = np.dot(strategy_player2, game_matrix[:, :, 1])
        best_response_player1 = np.argmax(expected_payoffs_player1)

        # Update strategy of player 1
        strategy_player1[best_response_player1] += 1

        # Update cumulative payoffs
        cum_payoffs_player1 += expected_payoffs_player1
        cum_payoffs_player2 += expected_payoffs_player2

    # Calculate average payoffs per strategy for each player
    avg_payoffs_player1 = cum_payoffs_player1 / num_iterations
    avg_payoffs_player2 = cum_payoffs_player2 / num_iterations

  # Determine if it doesn't matter which player starts
    if np.allclose(avg_payoffs_player1, avg_payoffs_player2[:, None], atol=1e-6):  # Compare each player's strategy individually
       return "Both Players", strategy_player1, strategy_player2

    # Determine which player has the higher average payoff
    if np.max(avg_payoffs_player1) > np.max(avg_payoffs_player2):
        starting_player = "Row Player (Player 1)"
        starting_strategy = np.argmax(avg_payoffs_player1)
    else:
        starting_player = "Column Player (Player 2)"
        starting_strategy = np.argmax(avg_payoffs_player2)

    return starting_player, starting_strategy

def get_game_matrix():
    rows = int(input("Enter the number of rows in the game matrix: "))
    cols = int(input("Enter the number of columns in the game matrix: "))
    game_matrix = np.zeros((rows, cols, 2))
    for i in range(rows):
        for j in range(cols):
            print(f"Enter the profits for cell ({i},{j}):")
            p1_profit = int(input("Profit for Row Player (Player 1): "))
            p2_profit = int(input("Profit for Column Player (Player 2): "))
            game_matrix[i][j] = [p1_profit, p2_profit]
    return game_matrix

# Example usage:
if __name__ == "__main__":
    game_matrix = get_game_matrix()
    if find_nash_equilibrium(game_matrix):
        starting_player, starting_strategy = choose_starting_player(game_matrix)
        if starting_player == "Both Players":
            print("\n")
            print("Both players can start. Strategies for both players are:")
            print(f"Row Player (Player 1): {starting_strategy[0]}")
            print(f"Column Player (Player 2): {starting_strategy[1]}")
        else:
            print("\n")
            print(f"Starting Player: {starting_player}")
            print(f"Starting Strategy Index for {starting_player}: {starting_strategy}")
    else:
        print("\n")
        print("This game does not have equilibrium.")